In [ ]:
git clone https://github.com/RasaHQ/rasa_nlu.git
cd rasa_nlu
pip install -r requirements.txt
pip install -e .

pip install spacy
python -m spacy download en_core_web_sm
python -m spacy download en

In [1]:
import sqlite3
conn = sqlite3.connect('telepot_users.db')
c = conn.cursor()

# c.execute("DROP TABLE users")

c.execute("CREATE TABLE IF NOT EXISTS users(first_name text, last_name text, id int, newest_message_id int, PRIMARY KEY (id))")

# c.execute("SELECT id FROM users")
# print(c.fetchall())

In [2]:
import requests

bot_token = "859640460:AAGw-HCQs9gpLR5G5OjcRAwXd7Og9a-d0oU"  
bot_url = "https://api.telegram.org/bot"

users_id = []
c.execute("SELECT id FROM users")
for i in c.fetchall():
    users_id.append(i[0])

users = []
c.execute("SELECT * FROM users")
for i in c.fetchall():
    user = {'first_name': i[0], 'last_name': i[1], 'id': i[2], 'newest_message_id': i[3]}
    users.append(user)

    
params = {}

#test
query = "INSERT INTO users(first_name, last_name, id, newest_message_id) VALUES('{0}', '{1}', '{2}', {3})".format('test',
                                                                                                                 'root',
                                                                                                                 0,
                                                                                                                 0)
c.execute(query)
c.execute("commit")

#Delete test data
query = "DELETE FROM users WHERE id=0"
c.execute(query)



In [3]:
def sendText(chat_id, message):
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + str(chat_id) + '&parse_mode=Markdown&text=' + message

    response = requests.get(send_text)

    return response.json()


# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)


from iexfinance.stocks import Stock
token = "sk_adf7c15b0a0947b0b5de15437773e2bf"


def replyMessage(id, message):
    if 'text' in message:
        output = interpreter.parse(message['text'])
        intent = output['intent']['name']
        if intent == 'greet':
            sendText(id, "Hello!")
            sendText(id, "I can search the stock information for you")
            sendText(id, "you could enquire me specific stock price, trading volume and company market capitalization")
        elif intent == 'goodbye':
            sendText(id, "see you!")  
        elif 'stock_search' in intent:
            if intent == "stock_search":
                if params.get('intent'):
                    if not params.get('company'):
                        params['company'] = output["entities"][0]["value"]
                        
                    s = Stock(params['company'], token=token)
                    s = s.get_quote()
                    if params['intent'] == "stock_search_price":
                        sendText(id, "{0} latest price is {1}".format(params['company'], s['latestPrice']))
                    elif params['intent'] == "stock_search_marketCap":
                        sendText(id, "market capitalization is {0}".format(s['marketCap']))
                    elif params['intent'] == "stock_search_avgTotalVolume":
                        sendText(id, "The average total volume of {0} is {1}".format(params['company'], s['avgTotalVolume']))
                    params.pop('intent')
                    params.pop('company')
                else:        
                    sendText(id, "Ok, what you want to know about? Stock price, market capitalization or trading volume")
            else:
                params['intent'] = intent
                for entity in output['entities']:
                    params[entity['entity']] = str(entity["value"])
                                    
                if params.get('company'):
                    m = {'text': params.get('company')}
                    replyMessage(id, m)
                    return
                sendText(id, "OK, please enter the company name or stock code")
          
        

/Users/Jerry/anaconda3/lib/python3.7/site-packages/rasa_nlu/training_data/training_data.py:176: UserWarning: Intent 'None' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/Jerry/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Jerry/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Jerry/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Jerry/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted sam

In [4]:
def saveUser(message):
        users_id.append(message['from']['id'])                                                        
        new_user = {'first_name': message['from']['first_name'],                                       
                    'last_name': message['from']['last_name'],                                        
                    'id': message['from']['id'],                                                      
                    'newest_message_id': message['message_id']                                        
                    }
        users.append(new_user)
        query = "INSERT INTO users(first_name, last_name, id, newest_message_id) VALUES('{0}', '{1}', '{2}', {3})".format(new_user['first_name'],
                                                                                                                 new_user['last_name'],
                                                                                                                 new_user['id'],
                                                                                                                 new_user['newest_message_id'])
        c.execute(query)
        c.execute("commit")

In [5]:
def getMe():                                                             
    return requests.get(bot_url + bot_token + '/getMe').json()           
                                                                         
                                                                         
def getUpdates():                                                        
    reply = requests.get(bot_url + bot_token + '/getUpdates').json()     
    return reply                                                         
                                                                         

In [ ]:
import time

print("running...")
while 1:
    buffer = getUpdates()['result']

    if buffer:                                                                                                                                                                                                          
        for message in buffer:                                                                                
            message = message['message']                                                                      
            user_id = message['from']['id']                                                                   
            if user_id not in users_id:

                saveUser(message)                                   
                sendText(new_user['id'], 'Hello ' + new_user['fist_name'])                                  
                print("A new user come in")

            else:                                                                                             
                for user in users:
                    if user['id'] == user_id:
                        if user.get('newest_message_id') < message['message_id']:                             
                            user['newest_message_id'] = message['message_id']
                            c.execute('UPDATE users SET newest_message_id = {0} WHERE id = {1}'.format(message['message_id'], user_id))
                            replyMessage(user['id'], message)                                                                                                                                                                   
    
    time.sleep(3)


running...
